In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..','..','..'))
import config, utils

# Create a Spark session
spark = utils.create_spark_session("playerStats", {
    'spark.executor.memory': '4g',
    'spark.executor.cores': '6',
})

fielding_data = utils.load_data(spark,config.PROCESSED_DATA_DIR, 'fielding_data.csv')
bowling_data = utils.load_data(spark,config.PROCESSED_DATA_DIR, 'bowling_data.csv')
batting_data = utils.load_data(spark,config.PROCESSED_DATA_DIR, 'batting_data.csv')

fielding_data.show(5)

/home/ravikumar/miniconda3/envs/t20i/lib/python3.12/site-packages/airflow/configuration.py:859 FutureWarning: section/key [core/sql_alchemy_conn] has been deprecated, you should use[database/sql_alchemy_conn] instead. Please update your `conf.get*` call to use the new name

[2024-11-19T03:52:48.209+0530] {utils.py:12} INFO - Creating Spark session.


your 131072x1 screen size is bogus. expect trouble
24/11/19 03:52:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/19 03:52:56 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


[2024-11-19T03:52:57.126+0530] {utils.py:30} INFO - Spark session created successfully.
[2024-11-19T03:52:57.132+0530] {utils.py:39} INFO - Loading data from fielding_data.csv.


[2024-11-19T03:53:12.021+0530] {utils.py:39} INFO - Loading data from bowling_data.csv.


[2024-11-19T03:53:13.294+0530] {utils.py:39} INFO - Loading data from batting_data.csv.


+---------+------------+---------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|player_id|      Player|  Country| Season|Cumulative Mat|Cumulative Inns|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+---------+------------+---------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
| b8d490fd|    AJ Finch|Australia|2016/17|            28|           28.0|           7.0|          7.0|          0.0|          0.25|
| b970a03f|   M Klinger|Australia|2016/17|             0|            0.0|           0.0|          0.0|          0.0|           0.0|
| 12b610c2|     TM Head|Australia|2016/17|             4|            4.0|           2.0|          2.0|          0.0|           0.5|
| 32198ae0|MC Henriques|Australia|2016/17|             6|            6.0|           2.0|          2.0|          0.0|          0.33|
| ff1e12a0|   AJ Turner|Australia|2016/17|             0|            0.0|   

In [2]:
# Check for nulls in the data
def check_nulls(df):
    nulls = 0
    for col in df.columns:
        nulls+= df.filter(df[col].isNull()).count()
    print("Total nulls in the dataframe: ", nulls)

check_nulls(batting_data)
check_nulls(bowling_data)
check_nulls(fielding_data)

Total nulls in the dataframe:  0
Total nulls in the dataframe:  0
Total nulls in the dataframe:  0


In [3]:
print(batting_data.columns,batting_data.count())
print(bowling_data.columns,bowling_data.count())
print(fielding_data.columns,fielding_data.count())

['Player', 'Country', 'Season', 'Cum Mat Total', 'Cum Inns Total', 'Cum Runs Total', 'Cum Batting Ave', 'Cum SR'] 14142
['player_id', 'Player', 'Country', 'Season', 'Cumulative Mat', 'Cumulative Inns', 'Cumulative Overs', 'Cumulative Bowling Runs', 'Cumulative Wkts', 'Cumulative Econ'] 50868
['player_id', 'Player', 'Country', 'Season', 'Cumulative Mat', 'Cumulative Inns', 'Cumulative Dis', 'Cumulative Ct', 'Cumulative St', 'Cumulative D/I'] 50868


In [10]:
playerdata = batting_data.join(bowling_data, on=['Player',"Country","Season"], how='inner').join(fielding_data, on=['Player',"Country","Season"], how='inner')\
    .drop('Cumulative Mat','Cumulative Inns')
playerdata = playerdata.dropDuplicates([ 'Player', 'Country', 'Season', 'player_id']).drop('player_id')
print(playerdata.count())
playerdata.show(5)

12012


+---------------+---------+-------+-------------+--------------+--------------+---------------+------+------------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
|         Player|  Country| Season|Cum Mat Total|Cum Inns Total|Cum Runs Total|Cum Batting Ave|Cum SR|  Cumulative Overs|Cumulative Bowling Runs|Cumulative Wkts|Cumulative Econ|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+---------------+---------+-------+-------------+--------------+--------------+---------------+------+------------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
|      HG Munsey| Scotland|2016/17|           13|            11|           190|          22.05|140.84|               0.0|                    0.0|            0.0|            0.0|           4.0|          4.0|          0.0|          0.36|
|   LD Chandimal|Sri Lanka|2018/19|           53|       

In [11]:
playerdata.sort(["Player","Country"]).show()

+-----------+--------------+-------+-------------+--------------+--------------+---------------+------+------------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
|     Player|       Country| Season|Cum Mat Total|Cum Inns Total|Cum Runs Total|Cum Batting Ave|Cum SR|  Cumulative Overs|Cumulative Bowling Runs|Cumulative Wkts|Cumulative Econ|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+-----------+--------------+-------+-------------+--------------+--------------+---------------+------+------------------+-----------------------+---------------+---------------+--------------+-------------+-------------+--------------+
| A Ahmadhel|      Bulgaria|   2023|            9|             7|            31|           4.43| 63.42| 17.40000009536743|                  155.0|            6.0|            9.3|           0.0|          0.0|          0.0|           0.0|
| A Ahmadhel|      Bulgaria|2020/21|            4|  

In [12]:
utils.spark_save_data(playerdata,config.PROCESSED_DATA_DIR, 'player_stats.csv')
spark.stop()

[2024-11-19T03:56:26.749+0530] {utils.py:57} INFO - Successfully wrote data to /usr/ravi/t20/data/2_processedData/player_stats.csv
